In [1]:
# Imports
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import numpy as np

# Farbdefinitionen
party_colors = {
    "CDU/CSU": "#000000",
    "AfD": "#0056FF",
    "SPD": "#E3000F",
    "B90/GRÜNE": "#1FA12E",
    "LINKE": "#A3488A",
    "BSW": "#D77AFF",
    "FDP": "#FFED00",
}

label_colors = {
    "Zustimmung": "#1FA12E",
    "Ablehnung": "#E3000F",
    "Neutral": "#B0B0B0"
}

# Daten laden
DATA_PATH1 = r"E:\Final_Github_1\MasterThesis_final\data\processed\preprocessing\mirror\mirror_llama_final.csv"
DATA_PATH2 = r"E:\Final_Github_1\MasterThesis_final\data\processed\preprocessing\augmentation\augmentation_llama_final.csv"

df_mirror = pd.read_csv(DATA_PATH1, sep=";", encoding="utf-8-sig", dtype={"timestamp": str})
df_aug    = pd.read_csv(DATA_PATH2, sep=";", encoding="utf-8-sig", dtype={"timestamp": str})

# Partei-Mapping
cdu_variants = ["CDU", "CSU", "CDU/CSU"]
for df in [df_mirror, df_aug]:
    if "party" in df.columns:
        df["party"] = df["party"].replace(cdu_variants, "CDU/CSU")

df_mirror["source"] = "Mirror"
df_aug["source"]    = "Augmentation"

# Daten kombinieren
df_all = pd.concat([df_mirror, df_aug], ignore_index=True)

# Export-Pfad für alle Tabellen/Figuren aus Kapitel 6.1
EXPORT_PATH = Path(r"E:\Final_Github_1\MasterThesis_final\data\analysis\Descriptive_Analyis_6_1")
EXPORT_PATH.mkdir(parents=True, exist_ok=True)

# Übersicht
print("Mirror:", df_mirror.shape, "| Augmentation:", df_aug.shape, "| Gesamt:", df_all.shape)
print("Parteien:", df_all['party'].unique())
print("Labels:", df_all['pred'].unique())
print("Unique Users:", df_all['username'].nunique())
print("Export Path:", EXPORT_PATH)



Mirror: (772378, 15) | Augmentation: (493426, 15) | Gesamt: (1265804, 15)
Parteien: ['B90/GRÜNE' 'AfD' 'BSW' 'LINKE' 'CDU/CSU' 'FDP' 'SPD' nan]
Labels: ['Ablehnung' 'Neutral' 'Zustimmung']
Unique Users: 149607
Export Path: E:\Final_Github_1\MasterThesis_final\data\analysis\Descriptive_Analyis_6_1


In [2]:
# Anzahl Tweets pro Handle (username) berechnen
tweet_counts = (
    df_all.groupby("X_handle")
    .size()
    .reset_index(name="tweet_count")
    .sort_values("tweet_count", ascending=False)
)

# Vorschau
print(tweet_counts.head(20))

# Optional: Als CSV exportieren für späteres Join mit den Account-Listen
tweet_counts.to_csv(EXPORT_PATH / "tweet_counts_per_handle.csv", index=False, encoding="utf-8-sig")
print("Exported:", EXPORT_PATH / "tweet_counts_per_handle.csv")


            X_handle  tweet_count
60    _FriedrichMerz       145597
41     Markus_Soeder       100905
100     roberthabeck        81612
2       Alice_Weidel        76429
70               cdu        60396
69         c_lindner        58102
75       die_gruenen        53739
61               afd        39046
0          ABaerbock        38370
80               fdp        31618
102            spdde        29618
74               csu        28567
71          cducsubt        27980
28   Karl_Lauterbach        27708
51      Ricarda_Lang        26204
88         jensspahn        23416
19    GoeringEckardt        22853
32           KrahMax        19598
27    JuliaKloeckner        15828
45        OlafScholz        15374
Exported: E:\Final_Github_1\MasterThesis_final\data\analysis\Descriptive_Analyis_6_1\tweet_counts_per_handle.csv


In [3]:
import pandas as pd
from pathlib import Path

# Pfad anpassen
LIST_PATH = Path(r"E:\Final_Github_1\MasterThesis_final\data\raw\list")

# Dateien laden
top3_path = LIST_PATH / "top3_per_party.csv"
top10_path = LIST_PATH / "top10_per_party.csv"
counts_path = LIST_PATH / "tweet_counts_per_handle.csv"

top3 = pd.read_csv(top3_path, sep=";", encoding="utf-8-sig", dtype=str)
top10 = pd.read_csv(top10_path, sep=";", encoding="utf-8-sig", dtype=str)
tweet_counts = pd.read_csv(counts_path, encoding="utf-8-sig", dtype=str)

# Handles normalisieren (kleinbuchstaben, @ entfernen, whitespaces trimmen)
for df in [top3, top10]:
    df["handle"] = df["handle"].str.lower().str.lstrip("@").str.strip()

tweet_counts.rename(columns={"X_handle": "handle"}, inplace=True)
tweet_counts["handle"] = tweet_counts["handle"].str.lower().str.lstrip("@").str.strip()
tweet_counts["tweet_count"] = pd.to_numeric(tweet_counts["tweet_count"], errors="coerce")

# Merge Counts
top3_merged = top3.merge(tweet_counts, on="handle", how="left")
top10_merged = top10.merge(tweet_counts, on="handle", how="left")

# Fehlende Matches finden
missing_top3 = top3_merged[top3_merged["tweet_count"].isna()]["handle"].unique().tolist()
missing_top10 = top10_merged[top10_merged["tweet_count"].isna()]["handle"].unique().tolist()

print("🔎 Fehlende Matches – Top 3 Parteien:", missing_top3)
print("🔎 Fehlende Matches – Top 10 Kandidaten:", missing_top10)

# Optional: Ergebnisse abspeichern
top3_merged.to_csv(LIST_PATH / "top3_with_counts.csv", sep=";", index=False, encoding="utf-8-sig")
top10_merged.to_csv(LIST_PATH / "top10_with_counts.csv", sep=";", index=False, encoding="utf-8-sig")

print("✅ Dateien gespeichert als top3_with_counts.csv und top10_with_counts.csv")


🔎 Fehlende Matches – Top 3 Parteien: []
🔎 Fehlende Matches – Top 10 Kandidaten: ['brihasselmann']
✅ Dateien gespeichert als top3_with_counts.csv und top10_with_counts.csv


In [4]:
# Volume Distribution 

def make_volume_table(df, label):
    table = (
        df.groupby("party")
        .agg(
            Tweets=("tweet_text", "count"),
            Tokens=("tweet_text", lambda x: x.str.split().str.len().sum()),
            Avg_Tweet_Length=("tweet_text", lambda x: x.str.split().str.len().mean())
        )
        .reset_index()
    )

    total_tweets = table["Tweets"].sum()
    table["%"] = (table["Tweets"] / total_tweets * 100).round(1)

    party_order = ["CDU/CSU", "SPD", "B90/GRÜNE", "FDP", "AfD", "LINKE", "BSW"]
    table = table.set_index("party").reindex(party_order).reset_index()

    total_row = pd.DataFrame({
        "party": ["Total"],
        "Tweets": [total_tweets],
        "%": [100.0],
        "Tokens": [table["Tokens"].sum()],
        "Avg_Tweet_Length": [df["tweet_text"].str.split().str.len().mean()]
    })
    table = pd.concat([table, total_row], ignore_index=True)
    table["source"] = label
    return table

# Tabellen erstellen
volume_mirror = make_volume_table(df_mirror, "Mirror")
volume_aug    = make_volume_table(df_aug, "Augmentation")
volume_all    = make_volume_table(df_all, "Combined")

# Export als einzelne CSVs
out_file_mirror = EXPORT_PATH / "volume_distribution_mirror.csv"
out_file_aug    = EXPORT_PATH / "volume_distribution_augmentation.csv"
out_file_all    = EXPORT_PATH / "volume_distribution_combined.csv"

volume_mirror.to_csv(out_file_mirror, sep=";", encoding="utf-8-sig", index=False)
volume_aug.to_csv(out_file_aug, sep=";", encoding="utf-8-sig", index=False)
volume_all.to_csv(out_file_all, sep=";", encoding="utf-8-sig", index=False)

print("CSVs gespeichert:")
print("Mirror:", out_file_mirror)
display(volume_mirror)
print("Augmentation:", out_file_aug)
display(volume_aug)
print("Combined:", out_file_all)
display(volume_all)



CSVs gespeichert:
Mirror: E:\Final_Github_1\MasterThesis_final\data\analysis\Descriptive_Analyis_6_1\volume_distribution_mirror.csv


,party,Tweets,Tokens,Avg_Tweet_Length,%,source
0,CDU/CSU,276858,5277183,19.060973,35.8,Mirror
1,SPD,75322,1421679,18.874685,9.8,Mirror
2,B90/GRÜNE,185869,3307294,17.793683,24.1,Mirror
3,FDP,80685,1552535,19.241928,10.4,Mirror
4,AfD,88938,1655807,18.617543,11.5,Mirror
5,LINKE,31873,595038,18.669030,4.1,Mirror
6,BSW,32833,655584,19.967228,4.3,Mirror
7,Total,772378,14465120,18.728032,100.0,Mirror


Augmentation: E:\Final_Github_1\MasterThesis_final\data\analysis\Descriptive_Analyis_6_1\volume_distribution_augmentation.csv


,party,Tweets,Tokens,Avg_Tweet_Length,%,source
0,CDU/CSU,178574,3396486,19.020048,36.2,Augmentation
1,SPD,54273,982746,18.107457,11.0,Augmentation
2,B90/GRÜNE,92429,1624999,17.581051,18.7,Augmentation
3,FDP,58708,1112432,18.948559,11.9,Augmentation
4,AfD,72337,1301292,17.989300,14.7,Augmentation
5,LINKE,18297,306176,16.733672,3.7,Augmentation
6,BSW,18807,361371,19.214707,3.8,Augmentation
7,Total,493425,9085502,18.413124,100.0,Augmentation


Combined: E:\Final_Github_1\MasterThesis_final\data\analysis\Descriptive_Analyis_6_1\volume_distribution_combined.csv


,party,Tweets,Tokens,Avg_Tweet_Length,%,source
0,CDU/CSU,455432,8673669,19.044927,36.0,Combined
1,SPD,129595,2404425,18.553378,10.2,Combined
2,B90/GRÜNE,278298,4932293,17.723063,22.0,Combined
3,FDP,139393,2664967,19.118370,11.0,Combined
4,AfD,161275,2957099,18.335756,12.7,Combined
5,LINKE,50170,901214,17.963205,4.0,Combined
6,BSW,51640,1016955,19.693164,4.1,Combined
7,Total,1265803,23550622,18.605277,100.0,Combined


In [5]:
# Stance Distribution (Table + Plots)

from plotly.subplots import make_subplots
import plotly.express as px
import plotly.graph_objects as go
import numpy as np
import pandas as pd

STANCE_ORDER = ["Support", "Neutral", "Opposition"]
label_colors = {"Support": "green", "Neutral": "lightgrey", "Opposition": "red"}

def make_stance_table(df, label):
    mapping = {"Zustimmung": "Support", "Ablehnung": "Opposition", "Neutral": "Neutral"}
    stance_df = df.copy()
    stance_df["stance"] = stance_df["pred"].map(mapping).fillna(df["pred"])

    stance_counts = stance_df.groupby(["party", "stance"]).size().reset_index(name="count")
    stance_pivot = stance_counts.pivot_table(index="party", columns="stance", values="count", fill_value=0)
    stance_pivot["Total"] = stance_pivot.sum(axis=1)

    for col in ["Support", "Neutral", "Opposition"]:
        stance_pivot[f"{col}_share"] = (stance_pivot[col] / stance_pivot["Total"] * 100).round(2)

    stance_pivot["NSI"] = (
        (stance_pivot["Support"] - stance_pivot["Opposition"]) /
        stance_pivot["Total"].replace(0, np.nan)
    ) * 100

    # NSI-Werte bei max. 0 kappen
    stance_pivot["NSI"] = stance_pivot["NSI"].clip(upper=0)

    stance_pivot = stance_pivot.reset_index()
    stance_pivot["source"] = label
    return stance_pivot

def plot_stance_distribution(df, label):
    mapping = {"Zustimmung": "Support", "Ablehnung": "Opposition", "Neutral": "Neutral"}
    stance_df = df.copy()
    stance_df["stance"] = stance_df["pred"].map(mapping).fillna(df["pred"])

    stance_counts = stance_df.groupby(["party", "stance"]).size().reset_index(name="count")
    parties = sorted(stance_counts["party"].unique().tolist())
    full_index = pd.MultiIndex.from_product([parties, STANCE_ORDER], names=["party", "stance"])
    stance_counts = stance_counts.set_index(["party", "stance"]).reindex(full_index, fill_value=0).reset_index()

    stance_counts["percent"] = stance_counts.groupby("party")["count"].transform(
        lambda x: 100 * x / max(x.sum(), 1)
    )

    count_pivot = stance_counts.pivot_table(index="party", columns="stance", values="count", fill_value=0)
    count_pivot["NSI"] = (
        (count_pivot["Support"] - count_pivot["Opposition"]) /
        (count_pivot["Support"] + count_pivot["Opposition"] + count_pivot["Neutral"]).replace(0, np.nan)
    ) * 100

    # NSI-Werte bei max. 0 kappen
    net_df = count_pivot.reset_index()[["party", "NSI"]]
    net_df["NSI"] = net_df["NSI"].clip(upper=0)

    fig_bars = px.bar(
        stance_counts,
        x="party", y="percent",
        color="stance",
        text=stance_counts["percent"].round(1).astype(str) + "%",
        color_discrete_map=label_colors,
        category_orders={"stance": STANCE_ORDER},
    )

    fig = make_subplots(specs=[[{"secondary_y": True}]] )
    
    for tr in fig_bars.data:
        fig.add_trace(tr, secondary_y=False)

    fig.add_trace(
        go.Scatter(
            x=net_df["party"],
            y=net_df["NSI"],
            mode="markers+lines+text",
            name="NSI",
            text=[f"{v:+.1f}%" for v in net_df["NSI"]],
            textposition="top center",
            line=dict(color="purple", width=3),
            textfont=dict(color="purple"),
            hovertemplate="<b>%{x}</b><br>NSI: %{y:.1f}%<extra></extra>",
        ),
        secondary_y=True,
    )

    fig.update_traces(textposition="inside", selector=dict(type="bar"))
    fig.update_layout(
        barmode="stack",
        xaxis_title="Party",
        legend_title="Stance",
        title=f"Stance Distribution ({label})",
        width=1100,
        margin=dict(t=50, l=40, r=40, b=40),
    )

    fig.update_yaxes(title_text="Proportion (%)", range=[0, 100], secondary_y=False)
    fig.update_yaxes(title_text="NSI", range=[-100, 0], secondary_y=True)

    fig.add_hline(y=0, line_width=1, line_dash="dot", line_color="gray", secondary_y=True)

    fig.show()


# Tabellen erzeugen
stance_random = make_stance_table(df_mirror, "Random")  # ehemals Mirror → Random
stance_aug    = make_stance_table(df_aug, "Augmentation")
stance_all    = make_stance_table(df_all, "Combined")

stance_random.to_csv(EXPORT_PATH / "stance_distribution_random.csv", sep=";", encoding="utf-8-sig", index=False)
stance_aug.to_csv(EXPORT_PATH / "stance_distribution_augmentation.csv", sep=";", encoding="utf-8-sig", index=False)
stance_all.to_csv(EXPORT_PATH / "stance_distribution_combined.csv", sep=";", encoding="utf-8-sig", index=False)

print("Random:", EXPORT_PATH / "stance_distribution_random.csv")
display(stance_random)
print("Augmentation:", EXPORT_PATH / "stance_distribution_augmentation.csv")
display(stance_aug)
print("Combined:", EXPORT_PATH / "stance_distribution_combined.csv")
display(stance_all)

# Plots erzeugen
plot_stance_distribution(df_mirror, "Random")
plot_stance_distribution(df_aug, "Augmentation")
plot_stance_distribution(df_all, "Combined")



Random: E:\Final_Github_1\MasterThesis_final\data\analysis\Descriptive_Analyis_6_1\stance_distribution_random.csv


stance,party,Neutral,Opposition,Support,Total,Support_share,Neutral_share,Opposition_share,NSI,source
0,AfD,11758,58669,18511,88938,20.81,13.22,65.97,-45.152803,Random
1,B90/GRÜNE,20662,144347,20860,185869,11.22,11.12,77.66,-66.437652,Random
2,BSW,4990,24345,3498,32833,10.65,15.20,74.15,-63.494046,Random
3,CDU/CSU,30614,221229,25015,276858,9.04,11.06,79.91,-70.871710,Random
4,FDP,9569,60939,10177,80685,12.61,11.86,75.53,-62.913801,Random
5,LINKE,4426,23648,3799,31873,11.92,13.89,74.19,-62.275280,Random
6,SPD,8374,61002,5946,75322,7.89,11.12,80.99,-73.094182,Random


Augmentation: E:\Final_Github_1\MasterThesis_final\data\analysis\Descriptive_Analyis_6_1\stance_distribution_augmentation.csv


stance,party,Neutral,Opposition,Support,Total,Support_share,Neutral_share,Opposition_share,NSI,source
0,AfD,9003,46499,16835,72337,23.27,12.45,64.28,-41.008059,Augmentation
1,B90/GRÜNE,10420,71377,10632,92429,11.50,11.27,77.22,-65.720715,Augmentation
2,BSW,2678,13978,2151,18807,11.44,14.24,74.32,-62.886159,Augmentation
3,CDU/CSU,18867,142555,17152,178574,9.60,10.57,79.83,-70.224669,Augmentation
4,FDP,6776,43957,7975,58708,13.58,11.54,74.87,-61.289773,Augmentation
5,LINKE,2045,13880,2372,18297,12.96,11.18,75.86,-62.895557,Augmentation
6,SPD,5488,44759,4026,54273,7.42,10.11,82.47,-75.052052,Augmentation


Combined: E:\Final_Github_1\MasterThesis_final\data\analysis\Descriptive_Analyis_6_1\stance_distribution_combined.csv


stance,party,Neutral,Opposition,Support,Total,Support_share,Neutral_share,Opposition_share,NSI,source
0,AfD,20761,105168,35346,161275,21.92,12.87,65.21,-43.293753,Combined
1,B90/GRÜNE,31082,215724,31492,278298,11.32,11.17,77.52,-66.199541,Combined
2,BSW,7668,38323,5649,51640,10.94,14.85,74.21,-63.272657,Combined
3,CDU/CSU,49481,363784,42167,455432,9.26,10.86,79.88,-70.618007,Combined
4,FDP,16345,104896,18152,139393,13.02,11.73,75.25,-62.229811,Combined
5,LINKE,6471,37528,6171,50170,12.30,12.90,74.80,-62.501495,Combined
6,SPD,13862,105761,9972,129595,7.69,10.70,81.61,-73.914117,Combined


In [6]:
# Nutzer-Zuordnung & Stance-Distribution auf User-Ebene

# Zustimmung-Tweets
df_support = df_all[df_all["pred"] == "Zustimmung"]

# Zustimmung pro User und Partei
user_party_counts = (
    df_support.groupby(["username", "party"])
    .size()
    .reset_index(name="support_count")
)

# Nur User mit ≥3 Zustimmungen (ASI-Definition)
user_party_assigned = user_party_counts[user_party_counts["support_count"] >= 3]

# Anzahl zugewiesener User pro Partei
assigned_users = (
    user_party_assigned.groupby("party")["username"]
    .nunique()
    .reset_index(name="assigned_users")
)

# Tweets (alle Stance-Klassen) dieser User zählen
stance_counts = (
    df_all[df_all["username"].isin(user_party_assigned["username"])]
    .groupby(["party", "pred"])
    .size()
    .unstack(fill_value=0)
    .reset_index()
)

# Merge
user_table = assigned_users.merge(stance_counts, on="party", how="left")

# Total Support Tweets
user_table["total_support_tweets"] = (
    df_support[df_support["username"].isin(user_party_assigned["username"])]
    .groupby("party")["pred"]
    .count()
    .reindex(user_table["party"])
    .fillna(0)
    .astype(int)
    .values
)

# Durchschnittliche Support-Tweets pro User
user_table["avg_support_tweets_per_user"] = (
    user_table["total_support_tweets"] / user_table["assigned_users"]
).round(2)

# Prozentuale Shares pro Partei
total_by_party = user_table[["Zustimmung", "Neutral", "Ablehnung"]].sum(axis=1)
for col in ["Zustimmung", "Neutral", "Ablehnung"]:
    user_table[f"{col}_share"] = (user_table[col] / total_by_party * 100).round(2)

# Total-Zeile
total_row = pd.DataFrame({
    "party": ["TOTAL"],
    "assigned_users": [user_table["assigned_users"].sum()],
    "total_support_tweets": [user_table["total_support_tweets"].sum()],
    "Ablehnung": [user_table["Ablehnung"].sum()],
    "Neutral": [user_table["Neutral"].sum()],
    "Zustimmung": [user_table["Zustimmung"].sum()],
    "avg_support_tweets_per_user": [round(user_table["avg_support_tweets_per_user"].mean(), 2)],
    "Zustimmung_share": [round(user_table["Zustimmung"].sum() / total_by_party.sum() * 100, 2)],
    "Neutral_share": [round(user_table["Neutral"].sum() / total_by_party.sum() * 100, 2)],
    "Ablehnung_share": [round(user_table["Ablehnung"].sum() / total_by_party.sum() * 100, 2)],
})
user_table = pd.concat([user_table, total_row], ignore_index=True)

# Reihenfolge Parteien
party_order = ["AfD", "B90/GRÜNE", "BSW", "CDU/CSU", "FDP", "LINKE", "SPD", "TOTAL"]
user_table = user_table.set_index("party").reindex(party_order).reset_index()

# Export
out_file_users = EXPORT_PATH / "user_distribution_full.csv"
user_table.to_csv(out_file_users, sep=";", encoding="utf-8-sig", index=False)


display(stance_all)
# Nutzerbasis
total_users = df_all["username"].nunique()
total_assigned_unique = user_party_assigned["username"].nunique()
share_assigned_total = round(total_assigned_unique / total_users * 100, 2)

print("Verteilung der zugewiesenen User inkl. Stance Shares:")
print("CSV gespeichert:")
print("User Distribution:", out_file_users)
display(user_table)


print("\nNutzerbasis:")
print(f"- Gesamtzahl individueller User im Corpus: {total_users:,}")
print(f"- Davon zugewiesen (ASI >=3 Supports): {total_assigned_unique:,} ({share_assigned_total}%)")



stance,party,Neutral,Opposition,Support,Total,Support_share,Neutral_share,Opposition_share,NSI,source
0,AfD,20761,105168,35346,161275,21.92,12.87,65.21,-43.293753,Combined
1,B90/GRÜNE,31082,215724,31492,278298,11.32,11.17,77.52,-66.199541,Combined
2,BSW,7668,38323,5649,51640,10.94,14.85,74.21,-63.272657,Combined
3,CDU/CSU,49481,363784,42167,455432,9.26,10.86,79.88,-70.618007,Combined
4,FDP,16345,104896,18152,139393,13.02,11.73,75.25,-62.229811,Combined
5,LINKE,6471,37528,6171,50170,12.30,12.90,74.80,-62.501495,Combined
6,SPD,13862,105761,9972,129595,7.69,10.70,81.61,-73.914117,Combined


Verteilung der zugewiesenen User inkl. Stance Shares:
CSV gespeichert:
User Distribution: E:\Final_Github_1\MasterThesis_final\data\analysis\Descriptive_Analyis_6_1\user_distribution_full.csv


,party,assigned_users,Ablehnung,Neutral,Zustimmung,total_support_tweets,avg_support_tweets_per_user,Zustimmung_share,Neutral_share,Ablehnung_share
0,AfD,2867,27118,6392,15961,15961,5.57,32.26,12.92,54.82
1,B90/GRÜNE,2585,62743,10487,16183,16183,6.26,18.10,11.73,70.17
2,BSW,371,11198,2544,2660,2660,7.17,16.22,15.51,68.27
3,CDU/CSU,4202,112199,17787,24179,24179,5.75,15.68,11.54,72.78
4,FDP,1382,33449,6126,9038,9038,6.54,18.59,12.60,68.81
5,LINKE,541,10248,2079,3278,3278,6.06,21.01,13.32,65.67
6,SPD,576,31005,4679,4637,4637,8.05,11.50,11.60,76.90
7,TOTAL,12524,287960,50094,75936,75936,6.49,18.34,12.10,69.56



Nutzerbasis:
- Gesamtzahl individueller User im Corpus: 149,607
- Davon zugewiesen (ASI >=3 Supports): 10,025 (6.7%)


In [7]:
# Zustimmung-Tweets filtern
df_support = df_all[df_all["pred"] == "Zustimmung"]

# Zustimmung pro User und Partei zählen
user_party_counts = (
    df_support.groupby(["username", "party"])
    .size()
    .reset_index(name="support_count")
)

# Nur User mit ≥3 Zustimmungen (ASI-Definition)
user_party_assigned = user_party_counts[user_party_counts["support_count"] >= 3]

# Jeder User → Partei mit den meisten Zustimmungen (tie-break: erste Partei alphabetisch)
user_max_party = (
    user_party_assigned.sort_values(["username", "support_count"], ascending=[True, False])
    .drop_duplicates(subset=["username"], keep="first")
)

# Merge zurück zu df_all → nur die eindeutig zugewiesenen User
assigned_users_list = user_max_party["username"].tolist()
df_assigned = df_all[df_all["username"].isin(assigned_users_list)]

# Tweets (alle Stance-Klassen) dieser User zählen
stance_counts = (
    df_assigned.groupby(["party", "pred"])
    .size()
    .unstack(fill_value=0)
    .reset_index()
)

# Anzahl zugewiesener User pro Partei
assigned_users = (
    user_max_party.groupby("party")["username"]
    .nunique()
    .reset_index(name="assigned_users")
)

# Merge
user_table = assigned_users.merge(stance_counts, on="party", how="left")

# Total Support Tweets (eindeutig zugewiesene User)
user_table["total_support_tweets"] = (
    df_support[df_support["username"].isin(assigned_users_list)]
    .groupby("party")["pred"]
    .count()
    .reindex(user_table["party"])
    .fillna(0)
    .astype(int)
    .values
)

# Durchschnittliche Support-Tweets pro User
user_table["avg_support_tweets_per_user"] = (
    user_table["total_support_tweets"] / user_table["assigned_users"]
).round(2)

# Prozentuale Shares pro Partei
total_by_party = user_table[["Zustimmung", "Neutral", "Ablehnung"]].sum(axis=1)
for col in ["Zustimmung", "Neutral", "Ablehnung"]:
    user_table[f"{col}_share"] = (user_table[col] / total_by_party * 100).round(2)

# TOTAL-Zeile mit unique Nutzern
total_row = pd.DataFrame({
    "party": ["TOTAL"],
    "assigned_users": [user_max_party["username"].nunique()],
    "total_support_tweets": [user_table["total_support_tweets"].sum()],
    "Ablehnung": [user_table["Ablehnung"].sum()],
    "Neutral": [user_table["Neutral"].sum()],
    "Zustimmung": [user_table["Zustimmung"].sum()],
    "avg_support_tweets_per_user": [round(
        user_table["total_support_tweets"].sum() / user_max_party["username"].nunique(), 2
    )],
    "Zustimmung_share": [round(user_table["Zustimmung"].sum() / total_by_party.sum() * 100, 2)],
    "Neutral_share": [round(user_table["Neutral"].sum() / total_by_party.sum() * 100, 2)],
    "Ablehnung_share": [round(user_table["Ablehnung"].sum() / total_by_party.sum() * 100, 2)],
})
user_table = pd.concat([user_table, total_row], ignore_index=True)

# Reihenfolge Parteien
party_order = ["AfD", "B90/GRÜNE", "BSW", "CDU/CSU", "FDP", "LINKE", "SPD", "TOTAL"]
user_table = user_table.set_index("party").reindex(party_order).reset_index()

# Export
out_file_users = EXPORT_PATH / "user_distribution_unique_assignment.csv"
user_table.to_csv(out_file_users, sep=";", encoding="utf-8-sig", index=False)

print("CSV gespeichert:", out_file_users)
display(user_table)

# Nutzerbasis
total_users = df_all["username"].nunique()
total_assigned_unique = user_max_party["username"].nunique()
share_assigned_total = round(total_assigned_unique / total_users * 100, 2)

print("\nNutzerbasis:")
print(f"- Gesamtzahl individueller User im Corpus: {total_users:,}")
print(f"- Davon eindeutig zugewiesen (ASI >=3 Supports): {total_assigned_unique:,} ({share_assigned_total}%)")


CSV gespeichert: E:\Final_Github_1\MasterThesis_final\data\analysis\Descriptive_Analyis_6_1\user_distribution_unique_assignment.csv


,party,assigned_users,Ablehnung,Neutral,Zustimmung,total_support_tweets,avg_support_tweets_per_user,Zustimmung_share,Neutral_share,Ablehnung_share
0,AfD,2638,27118,6392,15961,15961,6.05,32.26,12.92,54.82
1,B90/GRÜNE,2063,62743,10487,16183,16183,7.84,18.10,11.73,70.17
2,BSW,301,11198,2544,2660,2660,8.84,16.22,15.51,68.27
3,CDU/CSU,3420,112199,17787,24179,24179,7.07,15.68,11.54,72.78
4,FDP,886,33449,6126,9038,9038,10.20,18.59,12.60,68.81
5,LINKE,441,10248,2079,3278,3278,7.43,21.01,13.32,65.67
6,SPD,276,31005,4679,4637,4637,16.80,11.50,11.60,76.90
7,TOTAL,10025,287960,50094,75936,75936,7.57,18.34,12.10,69.56



Nutzerbasis:
- Gesamtzahl individueller User im Corpus: 149,607
- Davon eindeutig zugewiesen (ASI >=3 Supports): 10,025 (6.7%)


In [8]:
# ============================================
# Distribution: Alle Tweets von eindeutig zugewiesenen Usern (Sender-Perspektive)
# ============================================

# 1. Zustimmung-Tweets filtern
df_support = df_all[df_all["pred"] == "Zustimmung"]

# 2. Zustimmung pro User und Partei zählen
user_party_counts = (
    df_support.groupby(["username", "party"])
    .size()
    .reset_index(name="support_count")
)

# 3. Nur User mit ≥3 Zustimmungen (ASI-Definition)
user_party_assigned = user_party_counts[user_party_counts["support_count"] >= 3]

# 4. Jeder User → Partei mit den meisten Zustimmungen
user_max_party = (
    user_party_assigned.sort_values(["username", "support_count"], ascending=[True, False])
    .drop_duplicates(subset=["username"], keep="first")
    .rename(columns={"party": "user_party"})
)

# 5. Merge: Alle Tweets dieser eindeutig zugewiesenen User
df_assigned = df_all.merge(user_max_party[["username", "user_party"]], on="username", how="inner")

# 6. Tweets (alle Stance-Klassen) dieser User zählen – nach Sender-Partei
stance_counts = (
    df_assigned.groupby(["user_party", "pred"])
    .size()
    .unstack(fill_value=0)
    .reset_index()
    .rename(columns={"user_party": "party"})
)

# 7. Anzahl zugewiesener User pro Partei
assigned_users = (
    user_max_party.groupby("user_party")["username"]
    .nunique()
    .reset_index(name="assigned_users")
    .rename(columns={"user_party": "party"})
)

# 8. Merge
user_table = assigned_users.merge(stance_counts, on="party", how="left")

# 9. Total Support Tweets (Sender-Perspektive)
user_table["total_support_tweets"] = (
    df_assigned[df_assigned["pred"] == "Zustimmung"]
    .groupby("user_party")["pred"]
    .count()
    .reindex(user_table["party"])
    .fillna(0)
    .astype(int)
    .values
)

# 10. Durchschnittliche Support-Tweets pro User
user_table["avg_support_tweets_per_user"] = (
    user_table["total_support_tweets"] / user_table["assigned_users"]
).round(2)

# 11. Prozentuale Shares pro Partei (alle Tweets = Ablehnung+Neutral+Zustimmung)
total_by_party = user_table[["Zustimmung", "Neutral", "Ablehnung"]].sum(axis=1)
for col in ["Zustimmung", "Neutral", "Ablehnung"]:
    user_table[f"{col}_share"] = (user_table[col] / total_by_party * 100).round(2)

# 12. TOTAL-Zeile
total_row = pd.DataFrame({
    "party": ["TOTAL"],
    "assigned_users": [user_max_party["username"].nunique()],
    "total_support_tweets": [user_table["total_support_tweets"].sum()],
    "Ablehnung": [user_table["Ablehnung"].sum()],
    "Neutral": [user_table["Neutral"].sum()],
    "Zustimmung": [user_table["Zustimmung"].sum()],
    "avg_support_tweets_per_user": [round(
        user_table["total_support_tweets"].sum() / user_max_party["username"].nunique(), 2
    )],
    "Zustimmung_share": [round(user_table["Zustimmung"].sum() / total_by_party.sum() * 100, 2)],
    "Neutral_share": [round(user_table["Neutral"].sum() / total_by_party.sum() * 100, 2)],
    "Ablehnung_share": [round(user_table["Ablehnung"].sum() / total_by_party.sum() * 100, 2)],
})
user_table = pd.concat([user_table, total_row], ignore_index=True)

# 13. Reihenfolge Parteien
party_order = ["AfD", "B90/GRÜNE", "BSW", "CDU/CSU", "FDP", "LINKE", "SPD", "TOTAL"]
user_table = user_table.set_index("party").reindex(party_order).reset_index()

# 14. Export
out_file_users = EXPORT_PATH / "user_distribution_sender_assignment.csv"
user_table.to_csv(out_file_users, sep=";", encoding="utf-8-sig", index=False)

print("CSV gespeichert:", out_file_users)
display(user_table)

# 15. Nutzerbasis
total_users = df_all["username"].nunique()
total_assigned_unique = user_max_party["username"].nunique()
share_assigned_total = round(total_assigned_unique / total_users * 100, 2)

print("\nNutzerbasis:")
print(f"- Gesamtzahl individueller User im Corpus: {total_users:,}")
print(f"- Davon eindeutig zugewiesen (ASI >=3 Supports): {total_assigned_unique:,} ({share_assigned_total}%)")


CSV gespeichert: E:\Final_Github_1\MasterThesis_final\data\analysis\Descriptive_Analyis_6_1\user_distribution_sender_assignment.csv


,party,assigned_users,Ablehnung,Neutral,Zustimmung,total_support_tweets,avg_support_tweets_per_user,Zustimmung_share,Neutral_share,Ablehnung_share
0,AfD,2638,58333,8538,17840,17840,6.76,21.06,10.08,68.86
1,B90/GRÜNE,2063,69159,12277,17152,17152,8.31,17.40,12.45,70.15
2,BSW,301,7688,1729,2195,2195,7.29,18.90,14.89,66.21
3,CDU/CSU,3420,114162,19054,27470,27470,8.03,17.10,11.86,71.05
4,FDP,886,23645,5207,6618,6618,7.47,18.66,14.68,66.66
5,LINKE,441,7337,1755,2671,2671,6.06,22.71,14.92,62.37
6,SPD,276,7637,1534,1990,1990,7.21,17.83,13.74,68.43
7,TOTAL,10025,287961,50094,75936,75936,7.57,18.34,12.10,69.56



Nutzerbasis:
- Gesamtzahl individueller User im Corpus: 149,607
- Davon eindeutig zugewiesen (ASI >=3 Supports): 10,025 (6.7%)


In [9]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import pandas as pd

def clean_timestamp_column(df):
    df["timestamp_clean"] = (
        df["timestamp"]
        .str.replace("·", "", regex=False)
        .str.strip()
    )
    df["timestamp_clean"] = pd.to_datetime(
        df["timestamp_clean"], errors="coerce", utc=True
    )
    return df

def plot_volume_and_stance(df, label):
    df = clean_timestamp_column(df)

    # ---- Volume counts ----
    df["month"] = df["timestamp_clean"].dt.to_period("M")
    df["month_start"] = df["month"].dt.to_timestamp()
    counts = df.groupby(["month_start", "party"]).size().reset_index(name="count")

    # ---- Stance counts ----
    mapping = {"Zustimmung": "Support", "Ablehnung": "Rejection", "Neutral": "Neutral"}
    df["stance"] = df["pred"].map(mapping).fillna(df["pred"])
    stance_counts = (
        df.groupby(["month_start", "party", "stance"])
        .size()
        .reset_index(name="count")
    )
    stance_pivot = stance_counts.pivot_table(
        index=["month_start", "party"], columns="stance", values="count", fill_value=0
    ).reset_index()
    stance_pivot["Total"] = stance_pivot[["Support","Neutral","Rejection"]].sum(axis=1)
    stance_pivot["NSI"] = (
        (stance_pivot["Support"] - stance_pivot["Rejection"]) /
        stance_pivot["Total"].replace(0,1)
    ) * 100

    # ---- Subplot ----
    fig = make_subplots(
        rows=2, cols=1,
        shared_xaxes=True,
        vertical_spacing=0.08,
        subplot_titles=(f"Monthly Tweet Volume ({label})",
                        f"Monthly Stance Development (NSI) – {label}")
    )

    # Collect parties to ensure same colors
    parties = sorted(df["party"].dropna().unique())

    # Add traces (both panels)
    for party in parties:
        color = party_colors.get(party, "black")

        # Volume
        vol_sub = counts[counts["party"] == party]
        fig.add_trace(
            go.Scatter(
                x=vol_sub["month_start"], y=vol_sub["count"],
                mode="lines",
                name=party,
                line=dict(width=2, color=color),
                legendgroup=party,
                showlegend=True  # show once
            ),
            row=1, col=1
        )

        # Stance (NSI)
        stance_sub = stance_pivot[stance_pivot["party"] == party]
        fig.add_trace(
            go.Scatter(
                x=stance_sub["month_start"], y=stance_sub["NSI"],
                mode="lines",
                name=party,
                line=dict(width=2, color=color),
                legendgroup=party,
                showlegend=False  # suppress duplicate legend
            ),
            row=2, col=1
        )

    # Election marker
    election_day = pd.to_datetime("2025-02-23")
    for i in [1, 2]:
        fig.add_vline(
            x=election_day, line=dict(color="black", dash="dot"),
            row=i, col=1
        )
    fig.add_annotation(
        x=election_day, xref="x",
        y=1.02, yref="paper",
        text="Federal Election 2025", showarrow=False, xanchor="left"
    )

    # Layout
    fig.update_layout(
        height=800, width=1100,
        legend_title="Party",
        margin=dict(l=60, r=40, t=80, b=60)
    )
    fig.update_xaxes(
        range=[pd.to_datetime("2024-09-01"), pd.to_datetime("2025-03-01")],
        dtick="M1", tickformat="%b\n%Y"
    )
    fig.update_yaxes(title_text="Tweets", row=1, col=1)
    fig.update_yaxes(title_text="Net Stance Index (NSI)", row=2, col=1)

    fig.show()

# ---- Run for each dataset ----
for df, label in zip([df_mirror, df_aug, df_all],
                     ["Random", "Augmentation", "Combined"]):
    plot_volume_and_stance(df, label)




C:\Users\Admin\AppData\Local\Temp\ipykernel_23948\156088882.py:20: UserWarning:

Converting to PeriodArray/Index representation will drop timezone information.



C:\Users\Admin\AppData\Local\Temp\ipykernel_23948\156088882.py:20: UserWarning:

Converting to PeriodArray/Index representation will drop timezone information.



C:\Users\Admin\AppData\Local\Temp\ipykernel_23948\156088882.py:20: UserWarning:

Converting to PeriodArray/Index representation will drop timezone information.



In [10]:
# Top-3 Politiker pro Partei (Shares)

# Exclude-Liste (Partei-/Org-Accounts)
exclude_names_clean = sorted(set(n.strip().lower() for n in [
    "spdde", "spdbt", "jusos",
    "cdu", "cducsubt", "junge_union",
    "csu",
    "gruene_jugend", "die_gruenen", "gruenebundestag",  
    "fdp", "fdpbt", "fdp_nrw",
    "afd", "afdimbundestag", "afdberlin",
    "dielinke", "linksfraktion", "dielinkeberlin"
]))

# Helper: formatiert Namen "Nachname, Vorname"
def format_name(raw_name: str) -> str:
    s = str(raw_name)
    if "," in s:
        parts = [p.strip() for p in s.split(",")]
        last = parts[0]
        first = parts[1].split()[0] if len(parts) > 1 else ""
        return f"{last}, {first}" if first else last
    return s.strip()

df = df_all.copy()
df["name_lc"] = df["name"].astype(str).str.lower()
df["party"] = df["party"].fillna("Unbekannt")

# Masken: excluded (Partei/Org) vs. included (Politiker)
excluded_mask = df["name_lc"].isin(exclude_names_clean)
included_mask = ~excluded_mask
total_tweets = len(df)

# Gesamt pro Partei (nur Politiker)
included_df = df.loc[included_mask, ["party", "name"]].copy()
party_included_totals = (
    included_df.groupby("party").size().reset_index(name="included_total")
)

# Counts pro Politiker
counts = (
    included_df.groupby(["party", "name"])
    .size().reset_index(name="tweets")
)

# Top-3 pro Partei
top3_per_party = (
    counts.sort_values(["party", "tweets"], ascending=[True, False])
    .groupby("party", group_keys=False).head(3)
)

# Summen der Top-3
top3_sums = (
    top3_per_party.groupby("party")["tweets"].sum().reset_index(name="top3_sum")
)

# Shares berechnen
res = party_included_totals.merge(top3_sums, on="party", how="left").fillna({"top3_sum": 0})
res["share_all"] = (res["top3_sum"] / total_tweets * 100).round(2)
res["share_party"] = (res["top3_sum"] / res["included_total"] * 100).round(2)

# Namen der Top-3 je Partei sammeln
name_lists = (
    top3_per_party.assign(name_fmt=top3_per_party["name"].map(format_name))
    .groupby("party")["name_fmt"].apply(lambda s: ", ".join(s.tolist()))
    .reset_index(name="Top 3 Politicians")
)

# Finale Tabelle
final_table = (
    res.merge(name_lists, on="party", how="left")
      .loc[:, ["party", "Top 3 Politicians", "share_all", "share_party"]]
      .rename(columns={
          "party": "Party",
          "share_all": "Share of all tweets (%)",
          "share_party": "Share of party’s politician tweets (%)"
      })
      .sort_values("Party")
      .reset_index(drop=True)
)

# Export
out_file_top3 = EXPORT_PATH / "top3_politicians_per_party.csv"
final_table.to_csv(out_file_top3, sep=";", encoding="utf-8-sig", index=False)

print("CSV gespeichert:", out_file_top3)
display(final_table)


CSV gespeichert: E:\Final_Github_1\MasterThesis_final\data\analysis\Descriptive_Analyis_6_1\top3_politicians_per_party.csv


,Party,Top 3 Politicians,Share of all tweets (%),Share of party’s politician tweets (%)
0,AfD,"Weidel, Alice, Krah, Maximilian, von Storch, B...",8.56,92.08
1,B90/GRÜNE,"roberthabeck, Baerbock, Annalena, Lang, Ricarda",11.55,71.19
2,BSW,"Wagenknecht, Dr., De Masi, Fabio, Pürner, Frie...",2.53,62.13
3,CDU/CSU,"Merz, Joachim-Friedrich, Söder, Markus, Spahn,...",21.32,79.81
4,FDP,"Lindner, Christian, Buschmann, Dr., Strack-Zim...",6.78,84.32
5,LINKE,"Gysi, Gregor, Reichinnek, Heidi, Ramelow, Bodo",2.02,75.89
6,SPD,"Lauterbach, Dr., Scholz, Olaf, Faeser, Nancy",4.61,62.94


In [11]:
import pandas as pd
from pathlib import Path

# --- Paths ---
EXPORT_PATH = Path(r"E:\Final_Github_1\MasterThesis_final\data\analysis\Descriptive_Analyis_6_1")

# Make sure the directory exists
EXPORT_PATH.mkdir(parents=True, exist_ok=True)

# --- Helper Function to Aggregate Engagement ---
def engagement_summary_func(df: pd.DataFrame, name: str) -> pd.DataFrame:
    summary = (
        df.groupby("party")
        .agg(
            total_comments=("comments", "sum"),
            avg_comments=("comments", "mean"),
            total_retweets=("retweets", "sum"),
            avg_retweets=("retweets", "mean"),
            total_quotes=("quotes", "sum"),
            avg_quotes=("quotes", "mean"),
            total_likes=("likes", "sum"),
            avg_likes=("likes", "mean"),
            tweet_count=("comments", "count")
        )
        .reset_index()
    )

    # Gesamt-Engagement
    summary["total_engagement"] = (
        summary["total_comments"]
        + summary["total_retweets"]
        + summary["total_quotes"]
        + summary["total_likes"]
    )

    # Durchschnittliches Engagement pro Tweet
    summary["engagement_per_tweet"] = (
        summary["total_engagement"] / summary["tweet_count"]
    ).round(2)

    # Runden
    summary = summary.round({
        "avg_comments": 2,
        "avg_retweets": 2,
        "avg_quotes": 2,
        "avg_likes": 2
    })

    # Sortieren nach Gesamt-Engagement
    summary = summary.sort_values(by="total_engagement", ascending=False)

    # Exportieren
    out_file = EXPORT_PATH / f"engagement_summary_per_party_{name}.csv"
    summary.to_csv(out_file, sep=";", encoding="utf-8-sig", index=False)
    print(f"✅ CSV gespeichert: {out_file}")

    return summary


# --- Run for all datasets ---
summary_all = engagement_summary_func(df_all, "all")
summary_mirror = engagement_summary_func(df_mirror, "mirror")
summary_aug = engagement_summary_func(df_aug, "augmentation")

# Optional: Display in Jupyter
display(summary_all)
display(summary_mirror)
display(summary_aug)



✅ CSV gespeichert: E:\Final_Github_1\MasterThesis_final\data\analysis\Descriptive_Analyis_6_1\engagement_summary_per_party_all.csv
✅ CSV gespeichert: E:\Final_Github_1\MasterThesis_final\data\analysis\Descriptive_Analyis_6_1\engagement_summary_per_party_mirror.csv
✅ CSV gespeichert: E:\Final_Github_1\MasterThesis_final\data\analysis\Descriptive_Analyis_6_1\engagement_summary_per_party_augmentation.csv


,party,total_comments,avg_comments,total_retweets,avg_retweets,total_quotes,avg_quotes,total_likes,avg_likes,tweet_count,total_engagement,engagement_per_tweet
3,CDU/CSU,533604.0,1.17,530681.0,1.17,721558.0,1.58,380405.0,0.84,455372,2166248.0,4.76
1,B90/GRÜNE,337926.0,1.21,333204.0,1.20,456059.0,1.64,216820.0,0.78,278280,1344009.0,4.83
4,FDP,172402.0,1.24,163165.0,1.17,212867.0,1.53,122086.0,0.88,139380,670520.0,4.81
0,AfD,202716.0,1.26,175120.0,1.09,209107.0,1.30,67625.0,0.42,161263,654568.0,4.06
6,SPD,141858.0,1.09,134637.0,1.04,181206.0,1.40,86053.0,0.66,129583,543754.0,4.20
5,LINKE,64740.0,1.29,59865.0,1.19,55154.0,1.10,28084.0,0.56,50166,207843.0,4.14
2,BSW,66164.0,1.28,51771.0,1.00,61488.0,1.19,23314.0,0.45,51633,202737.0,3.93


,party,total_comments,avg_comments,total_retweets,avg_retweets,total_quotes,avg_quotes,total_likes,avg_likes,tweet_count,total_engagement,engagement_per_tweet
3,CDU/CSU,325112.0,1.17,322048.0,1.16,431584.0,1.56,239987.0,0.87,276816,1318731.0,4.76
1,B90/GRÜNE,222276.0,1.20,214325.0,1.15,294977.0,1.59,144144.0,0.78,185858,875722.0,4.71
4,FDP,93504.0,1.16,89115.0,1.10,111809.0,1.39,70677.0,0.88,80678,365105.0,4.53
0,AfD,108103.0,1.22,92759.0,1.04,106692.0,1.20,38041.0,0.43,88931,345595.0,3.89
6,SPD,85380.0,1.13,77053.0,1.02,104773.0,1.39,49070.0,0.65,75313,316276.0,4.20
2,BSW,41740.0,1.27,33068.0,1.01,41615.0,1.27,16891.0,0.51,32826,133314.0,4.06
5,LINKE,41851.0,1.31,38685.0,1.21,32687.0,1.03,19391.0,0.61,31870,132614.0,4.16


,party,total_comments,avg_comments,total_retweets,avg_retweets,total_quotes,avg_quotes,total_likes,avg_likes,tweet_count,total_engagement,engagement_per_tweet
3,CDU/CSU,208492.0,1.17,208633.0,1.17,289974.0,1.62,140418.0,0.79,178556,847517.0,4.75
1,B90/GRÜNE,115650.0,1.25,118879.0,1.29,161082.0,1.74,72676.0,0.79,92422,468287.0,5.07
0,AfD,94613.0,1.31,82361.0,1.14,102415.0,1.42,29584.0,0.41,72332,308973.0,4.27
4,FDP,78898.0,1.34,74050.0,1.26,101058.0,1.72,51409.0,0.88,58702,305415.0,5.20
6,SPD,56478.0,1.04,57584.0,1.06,76433.0,1.41,36983.0,0.68,54270,227478.0,4.19
5,LINKE,22889.0,1.25,21180.0,1.16,22467.0,1.23,8693.0,0.48,18296,75229.0,4.11
2,BSW,24424.0,1.30,18703.0,0.99,19873.0,1.06,6423.0,0.34,18807,69423.0,3.69
